## Load Library

In [1]:
import math
import time
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [2]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [3]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

df

,Name,X,Y
0,Depot,265,143
1,Parcel1,253,278
2,Point2,439,148
3,Parcel3,458,304
4,Point4,609,230
5,Parcel5,597,101


In [4]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [5]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def toString(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

In [6]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [7]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Create Point Array

In [8]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
name_arr = list(map(lambda x : x.name, pt_arr))
name_arr

['Depot', 'Parcel1', 'Point2', 'Parcel3', 'Point4', 'Parcel5']

## Create Edge Array

In [9]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2), 3)
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name) or ("Parcel" in pt.name and "Parcel" in pt2.name)or ("Parcel" in pt.name and "Depot" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_mat

[[inf, 135.532, inf, 251.336, inf, 334.646],
 [inf, inf, 226.927, inf, 359.221, inf],
 [174.072, 226.927, inf, 157.153, inf, 164.842],
 [inf, inf, 157.153, inf, 168.158, inf],
 [354.831, 359.221, inf, 168.158, inf, 129.557],
 [inf, inf, 164.842, inf, 129.557, inf]]

In [10]:
def gen_adj_mat(adj_mat, column_arr):
    tmp_adj_mat = []
    for row_ind in range(len(adj_mat)):
        if row_ind in column_arr:
            tmp_list = []
            for col_ind in column_arr:
                tmp_list.append(adj_mat[row_ind][col_ind])
            tmp_adj_mat.append(tmp_list)
            
    return tmp_adj_mat

In [11]:
gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])

[[inf, inf, 251.336, inf, 334.646],
 [174.072, inf, 157.153, inf, 164.842],
 [inf, 157.153, inf, 168.158, inf],
 [354.831, inf, 168.158, inf, 129.557],
 [inf, 164.842, inf, 129.557, inf]]

## Create Path

In [12]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"
    
    def __eq__(self, other):
        return self.path == other.path and self.cost == other.cost
    
    def getNodePath(self, name_arr):
        tmp_arr = list(map(lambda x : name_arr[x], self.path))
        return "->".join(tmp_arr)

## Create TSP Matrix

In [13]:
import itertools
import random
import sys

def held_karp(dists, column_arr, route_arr, point_arr, possible_combination_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset in possible_combination_arr:
        # Set bits for all nodes in this subset
        bits = 0
        for bit in subset:
            bits |= 1 << bit

        # Find the lowest cost to get to this subset
        for k in subset:
            prev = bits & ~(1 << k)

            res = []
            for m in subset:
                if m == 0 or m == k:
                    continue
                        
                # Check if Key Exist
                key = (prev, m)
                res.append((C[key][0] + dists[m][k], m))
            C[(bits, k)] = min(res)
                
    best_bits, best_opt, best_parent = 0, float('inf'), 0

    # Calculate optimal cost
    res = []
    for route_bits in route_arr:
        for k in point_arr:
            tmp_opt = C[(route_bits,k)][0] + dists[k][0]
            
            if tmp_opt <= best_opt:
                best_opt = tmp_opt
                best_parent = k
                best_bits = route_bits

    # Backtrack to find full path
    path = ["Depot"] * (2 * len(point_arr) + 2)
    for i in range(2 * len(point_arr), 0, -1):
        tmp_val = best_parent
        path[i] = column_arr[tmp_val]
        _, best_parent = C[(best_bits, tmp_val)]
        best_bits = best_bits & ~(1 << tmp_val)
        
    return best_opt, path

In [14]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

start = time.process_time()
route_arr = []
for arr in set(itertools.combinations(parcel_arr, len(point_arr))):
    tmp_arr = [*arr] + point_arr
    route_arr.append(sum([1 << elem for elem in tmp_arr]))
print(time.process_time() - start)

tmp_arr = []

start = time.process_time()
possible_combination_arr = []
for k in range(len(point_arr) + 1):
    for arr in set(itertools.combinations(parcel_arr, k)):
        for l in range(1, k + 1):
            for arr_2 in set(itertools.combinations(point_arr, l)):
                tmp_arr = [*arr] + [*arr_2]
                possible_combination_arr.append(tmp_arr)
                
print(possible_combination_arr)
print(time.process_time() - start)

0.0
[[1, 2], [1, 4], [3, 2], [3, 4], [5, 2], [5, 4], [1, 3, 2], [1, 3, 4], [1, 3, 2, 4], [3, 5, 2], [3, 5, 4], [3, 5, 2, 4], [1, 5, 2], [1, 5, 4], [1, 5, 2, 4]]
0.0


In [15]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# min_path = Path([], float('inf'))

# final_start = time.process_time()

# combo_set = set(itertools.combinations(parcel_arr, len(point_arr)))
# total, ind = len(combo_set), 1

# print(f"Total Combinations: {total}")
# for parcel_set in combo_set:
    
#     tmp_list = [0] + [*parcel_set] + point_arr
#     tmp_list = sorted(tmp_list)
    
#     tmp_adj_mat = gen_adj_mat(adj_mat, tmp_list)
    
#     tmp_list = list(map(lambda x : name_arr[x], tmp_list))
    
#     start = time.process_time()
#     val, best_path = held_karp(tmp_adj_mat, tmp_list)
#     print(f"Time Taken (Held-Karp): {time.process_time() - start}")
    
#     min_path = min(min_path, Path(best_path, val))
    
#     print(f"Progress: {round(ind / total * 100.0, 2)}%\n")
    
#     ind += 1
    
# print(f"Final Time Taken (Held-Karp): {time.process_time() - final_start}")
# print(min_path)

In [16]:
start = time.process_time()
val, best_path = held_karp(adj_mat, name_arr, route_arr, point_arr, possible_combination_arr)
print(Path(val, best_path))
print(time.process_time() - start)

KeyError: (10, 1)